In [3]:
from word2vectraining import *
import csv
import random as rd
from list_to_onehot import *

vocab_size = 23940
dim = 200
no_ofppl = 20
input_sentsize = 30
input_size_bot = dim*input_sentsize + no_ofppl

In [4]:
#word2vec

def word2vec(vocab_size = vocab_size,h_size = dim):
    model = ke.Sequential()
    model.add(ke.layers.Dense(h_size, input_shape = (vocab_size,)))
    model.add(ke.layers.Dense(vocab_size,activation = ke.activations.softmax))
    return model

In [5]:
#model creation
word2vec_model = word2vec()
word2vec_model.compile(loss="categorical_crossentropy",optimizer='adam', metrics=['accuracy'])

In [19]:
#useful functions

def shuffle_csv(path,target_path):
    with open(path,'r') as file:
        csvreader = csv.reader(file)
        header = next(csvreader)
        rows_list = []
        for row in csvreader:
            rows_list.append(row)
        rd.shuffle(rows_list)
        with open(target_path,'w', newline='') as outfile:
            csvwriter = csv.writer(outfile)
            csvwriter.writerow(header)
            csvwriter.writerows(rows_list)

def fetch_rows(start,end,csv_path): #[) kinda interval
    with open(csv_path,'r') as file:
        csvreader = csv.reader(file)
        header = next(csvreader)
        tup_lst = []
        pos = 0
        for row in csvreader:
            if pos >= start and pos < end:
                
                tup = (eval(row[1]),eval(row[2]))
                tup_lst.append(tup)
                
            pos += 1
    return tup_lst

fetch_rows(1,10,"shuffled_tupledata.csv")

def fetch_next_batch(prev_end,batch_size,csv_path):
    batch = fetch_rows(prev_end,prev_end+batch_size,csv_path)
    if len(batch) != batch_size:
        return None
    else:
        return batch

def evaluate_model(batch_size,data_total_size,model,csv_path):
    val_batch = fetch_rows(data_total_size-batch_size,data_total_size,csv_path)
    X_lst = []
    Y_lst = []
    dict_test = {}
    for tup in val_batch:
        x = convertInputToOneHotPercentages(tup[0],vocab_size)
        y = convertInputToOneHotPercentages(tup[1],vocab_size)
        if (type(x)==type(dict_test)):
            item1 = list(x.values())[0]
            X_lst.append(item1)
        else:
            X_lst.append(x)
        if (type(x)==type(dict_test)):
            item2 = list(x.values())[0]
            Y_lst.append(item2)
        else:
            Y_lst.append(y)
        
        
    X = np.array(X_lst)
    print(X.shape)
    Y = np.array(Y_lst)
    print(Y.shape)
    return model.evaluate(X,Y)

In [20]:
# training loop
import time

start = time.time()

no_of_epochs = 1
batch_size = 3000
data_total_size = 296917
tolerance = 10 #for early stopping

max_acc = 0
unchanged_epochs = 0
for epoch in range(1,no_of_epochs+1):
    print("Epoch: ",epoch)
    shuffle_csv("tupledata.csv","shuffled_tupledata.csv")
    prev_end = 0
    batch = []
    while batch != None:
        batch = fetch_next_batch(prev_end,batch_size,"shuffled_tupledata.csv")
        if batch == None:
            break
        train_on_tup_batch(batch,word2vec_model)
        prev_end = prev_end + batch_size
    acc = evaluate_model(batch_size,data_total_size,word2vec_model,"shuffled_tupledata.csv")
    print("Accuracy: ",acc)
    if max_acc >= acc:
        unchanged_epochs += 1
        if unchanged_epochs >= 10:
            break
    else:
        max_acc = acc

end = time.time()
print("Training Complete:")
print("Epochs: ",epoch)
print("Accuracy: ",acc)
print("Time Taken: ",end-start,"secs")



Epoch:  1
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.] ext from dict
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.] ext from dict
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.] ext from dict
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.] ext from dict
[0. 0. 0. ... 0. 0. 0.] ext from dict
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.] ext from dict
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.] ext from dict
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.] ext from dict
[0. 0. 0. ... 0. 0. 0.] ext from dict
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.] 

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3000,) + inhomogeneous part.

In [ ]:
#testing model

val_batch = fetch_rows(data_total_size-batch_size,batch_size)
X_lst = []
Y_lst = []
for tup in val_batch:
    x = convertInputToOneHotPercentages(tup[0])
    y = convertInputToOneHotPercentages(tup[1])
    X_lst.append(x)
    Y_lst.append(y)
X = np.array(X_lst)
Y = np.array(Y_lst)
word2vec_model.evaluate(X,Y)

In [12]:
dict1 = {}
print(type(dict1))

<class 'dict'>
